In [ ]:
!pip install openai google huggingface_hub google-genai gradio transformers torch bitsandbytes requests accelerate sentencepiece -q

In [ ]:
import os
import io
import sys
from openai import OpenAI
from google import genai
from IPython.display import Markdown, display, update_display
import gradio as gr
import subprocess
import platform
from google.colab import userdata
from transformers import AutoTokenizer, AutoModelForCausalLM, TextStreamer, BitsAndBytesConfig, TextStreamer
import torch

In [ ]:
openai_api_key = userdata.get('OPENAI_API_KEY')
openai = OpenAI(api_key=openai_api_key)

google_key = userdata.get('GOOGLE_API_KEY')

In [ ]:
system_message = "You are an assistant that reimplements Python code in high performance C++ for an M1 Mac. "
system_message += "Respond only with C++ code; use comments sparingly and do not provide any explanation other than occasional comments. "
system_message += "The C++ response needs to produce an identical output in the fastest possible time. Keep implementations of random number generators identical so that results match exactly."

In [ ]:
def user_prompt(python):
    user_prompt = "Rewrite this Python code in C++ with the fastest possible implementation that produces identical output in the least time. "
    user_prompt += "Respond only with C++ code; do not explain your work other than a few comments. "
    user_prompt += "Pay attention to number types to ensure no int overflows. Remember to #include all necessary C++ packages such as iomanip.\n\n"
    user_prompt += python
    return user_prompt

In [ ]:
def messages_for(python):
    return [
        {"role": "system", "content": system_message},
        {"role": "user", "content": user_prompt(python)}
    ]

In [ ]:
pi = """
import time

def calculate(iterations, param1, param2):
    result = 1.0
    for i in range(1, iterations+1):
        j = i * param1 - param2
        result -= (1/j)
        j = i * param1 + param2
        result += (1/j)
    return result

start_time = time.time()
result = calculate(100_000_000, 4, 1) * 4
end_time = time.time()

print(f"Result: {result:.12f}")
print(f"Execution Time: {(end_time - start_time):.6f} seconds")
"""

In [ ]:
hf_token = userdata.get('HF_TOKEN')

In [ ]:
NxCode = 'NTQAI/Nxcode-CQ-7B-orpo'

In [ ]:
quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_quant_type="nf4"
)

tokenizer = AutoTokenizer.from_pretrained(NxCode)
tokenizer.pad_token = tokenizer.eos_token
messages = messages_for(pi)

model = AutoModelForCausalLM.from_pretrained(
    NxCode,
    device_map="auto",
    quantization_config=quant_config
)

In [ ]:
import re

def generate_code_NxCode(model, tokenizer, messages):
    inputs = tokenizer.apply_chat_template(messages, return_tensors='pt',add_generation_prompt=False).to('cuda')
    streamer = TextStreamer(tokenizer, skip_prompt=True)
    outputs = model.generate(inputs, max_new_tokens=2000)
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # Extract C++ code block between ```cpp ... ```
    match = re.search(r"```cpp(.*?)```", response, re.DOTALL)
    if match:
        cpp_code = match.group(1).strip()
    else:
        cpp_code = "C++ code not found."

    return cpp_code


In [ ]:
cpp_code_NxCode = generate_code_NxCode(model, tokenizer, messages)
print(cpp_code_NxCode)

In [ ]:
def extract_cpp_code(python):
    messages = messages_for(python)
    inputs = tokenizer.apply_chat_template(messages, return_tensors='pt', add_generation_prompt=False).to('cuda')
    outputs = model.generate(inputs, max_new_tokens=2000)
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)

    match = re.search(r"```cpp(.*?)```", response, re.DOTALL)
    return match.group(1).strip() if match else "C++ code not found."

In [ ]:
extract_cpp_code(pi)

In [ ]:
with gr.Blocks() as ui :
  gr.Markdown('# Convert Python to C++')

  with gr.Row():
      python = gr.Textbox(label="Python code:", value=pi, lines=10)
      cpp = gr.Textbox(label="C++ code:", lines=10)

  with gr.Row():
      convert = gr.Button("Convert code")

  convert.click(extract_cpp_code, inputs=[python], outputs=[cpp])

ui.launch(share=True,debug=True)